In [1]:
%pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:

import mysql.connector

from tabulate import tabulate

# Connect to the MySQL database
connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="EymsToMmCKG1868.root",
    password="Hh15Cev73dkxuTLJ",
    database="test",
)

# Create a cursor object
mycursor = connection.cursor(buffered=True)

# Execute SQL queries
mycursor.execute("show databases")
for i in mycursor:
    print(i)
mycursor=connection.cursor(buffered=True)


('INFORMATION_SCHEMA',)
('PERFORMANCE_SCHEMA',)
('RedBus',)
('guvi',)
('joins',)
('mysql',)
('prac',)
('test',)


In [15]:
mycursor.execute("CREATE DATABASE RedBus")

In [19]:
#create a table in database in REDBUS

mycursor.execute("USE RedBus")

mycursor.execute("CREATE TABLE BusDetails (ID INT AUTO_INCREMENT PRIMARY KEY,\
                 State VARCHAR(255),\
                 Route VARCHAR(255),\
                 Link VARCHAR(255),\
                 Bus_Operator VARCHAR(255),\
                 Bus_Type VARCHAR(255),\
                 Bus_Departing_Time VARCHAR(255),\
                 Bus_Active_Time VARCHAR(255),\
                 Bus_Duration TIME,\
                 Bus_Price DECIMAL(10,2),\
                 Seats INT,\
                 Rating FLOAT)")





In [26]:
#moddify the name of column Bus_Active_Time to Bus_Arrival_Time

mycursor.execute("ALTER TABLE RedBus.BusDetails CHANGE Bus_Arrival_Time Bus_Reaching_Time TIME")

In [28]:
mycursor.execute("ALTER TABLE RedBus.BusDetails CHANGE Bus_Price Price DECIMAL (10,2)")

In [3]:
#change the datatype of Bus_duration from TIME to VARCHAR

mycursor.execute("ALTER TABLE RedBus.BusDetails CHANGE Bus_Duration Bus_Duration VARCHAR(255)")

In [4]:
import pandas as pd

#load the data into sql database table RedBus.BusDetails from CSV file

csv_file = r'C:\Users\geeth\OneDrive\Desktop\GUVI Projects\StreamlitRB\Guvi_RedBus.csv'

# Read the CSV file
df = pd.read_csv(csv_file)

# Insert data into the SQL table
for index, row in df.iterrows():
    mycursor.execute("""
        INSERT INTO RedBus.BusDetails (State, Route, Link, Bus_Operator, Bus_Type, Bus_Departing_Time, Bus_Reaching_Time, Bus_Duration, Price, Seats, Rating) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['State'], row['Route'], row['Link'], row['Bus_Operator'], row['Bus_Type'], row['Bus_Departing_Time'], row['Bus_Reaching_Time'], row['Bus_Duration'], row['Price'], row['Seats'], row['Rating']
    ))

# Commit the transaction
connection.commit()

In [3]:
#view the data in the database

mycursor.execute("SELECT * FROM RedBus.BusDetails")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

connection.commit()


+------+-----------+---------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------+--------------------------------------------+----------------------+---------------------+----------------+---------+---------+----------+
|   ID | State     | Route                                       | Link                                                                                                                                                                                                                            | Bus_Operator                                            | Bus_Type                                   | Bus_Departing_Time   | Bus_Reaching_Time   | Bus_Duration   |   Price |   Seats |   Rating |
|------+-----------+--

In [6]:
#show tables in the database

mycursor.execute("SHOW TABLES FROM RedBus")
for i in mycursor:
    print(i)

('BusDetails',)


In [7]:
#show the columns in the table

mycursor.execute("DESCRIBE RedBus.BusDetails")
for i in mycursor:
    print(i)

('ID', 'int(11)', 'NO', 'PRI', None, 'auto_increment')
('State', 'varchar(255)', 'YES', '', None, '')
('Route', 'varchar(255)', 'YES', '', None, '')
('Link', 'varchar(255)', 'YES', '', None, '')
('Bus_Operator', 'varchar(255)', 'YES', '', None, '')
('Bus_Type', 'varchar(255)', 'YES', '', None, '')
('Bus_Departing_Time', 'varchar(255)', 'YES', '', None, '')
('Bus_Reaching_Time', 'time', 'YES', '', None, '')
('Bus_Duration', 'varchar(255)', 'YES', '', None, '')
('Price', 'decimal(10,2)', 'YES', '', None, '')
('Seats', 'int(11)', 'YES', '', None, '')
('Rating', 'float', 'YES', '', None, '')


In [6]:
#create a streamlit application with the above data. Use state, price as filters. Then rating as slider. Put a button to filter the data based on the above filters. Display the data in a table format.

import streamlit as st
import pandas as pd

# Load the data

df = pd.read_sql_query("SELECT * FROM RedBus.BusDetails", connection)

# Create a sidebar for filters

st.sidebar.title("Filter Bus Data")
st.sidebar.subheader("Select filters")

state_filter = st.selectbox("State", df['State'].unique())
price_filter = st.slider("Price Range", min_value=df['Price'].min(), max_value=df['Price'].max(), value=(df['Price'].min(), df['Price'].max()))

filtered_df = df[(df['State'] == state_filter) & (df['Price'] >= price_filter[0]) & (df['Price'] <= price_filter[1])]

# Display the filtered data

st.dataframe(filtered_df)

# Create a button to filter data

if st.button("Filter"):
    filtered_df = df[(df['State'] == state_filter) & (df['Price'] >= price_filter[0]) & (df['Price'] <= price_filter[1])]
    st.dataframe(filtered_df)



C:\Users\geeth\AppData\Local\Temp\ipykernel_23276\3550588086.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM RedBus.BusDetails", connection)
2024-08-03 15:43:48.879 
  command:

    streamlit run C:\Users\geeth\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-08-03 15:43:48.895 Session state does not function when running a script without `streamlit run`
